In [325]:
import yfinance as yf
import pandas as pd
from numpy import linalg
import numpy as np

In [326]:
# define the ticker string.
tickerSymbol = ['MSFT', 'A', 'V', 'F', 'X', 'AA', 'JPM', 'XOM', 'BAC', 'GE', 'ABB', 'WMT', 'HPQ']

ticker = 'ACB'
tickerData = yf.Ticker(ticker)
tickerData = tickerData.history(period='1d', start='2018-1-1', end='2020-1-25')    
tickerData['TickString'] = ticker
tickerData

for tick in tickerSymbol:
    tickerDataRetrieve = yf.Ticker(tick)
    tickerDf = tickerDataRetrieve.history(period='1d', start='2019-1-1', end='2020-1-25')    
    tickerDf['TickString'] = tick
    tickerData = tickerData.append(tickerDf)

tickerData = tickerData.reset_index()
tickerData = tickerData[['Date', 'Close', 'TickString']]

In [327]:
# Convert to Pct_Change
tickerData = tickerData.set_index(['Date', 'TickString'])
tickerData = tickerData.groupby(level="TickString").pct_change(1)

In [328]:
tickerData.columns = ['Pct_Change']
tickerData = tickerData.reset_index()

In [329]:
# Convert from long to wide format
tickerD = pd.pivot_table(tickerData, index=["Date"], columns=['TickString'], values=["Pct_Change"])

In [330]:
# Remove any NaNs
tickerD = tickerD.dropna(axis=0)
tickerD.reset_index(drop=True, inplace=True)

In [331]:
# Demean the columns
tickerD = tickerD - tickerD.mean()

In [332]:
tickerD.head()

Pct_Change                                                    \
TickString          A        AA       ABB       ACB       BAC         F   
0           -0.038119  0.001634 -0.021109 -0.016539 -0.017076 -0.016299   
1            0.033500  0.081664  0.035108  0.018110  0.040055  0.037362   
2            0.019951  0.004457 -0.004466  0.010208 -0.002106  0.025021   
3            0.013297 -0.004348  0.013046 -0.041181 -0.003323  0.008385   
4            0.019811  0.031722  0.011177  0.086044  0.008457  0.041281   

                                                                        \
TickString        GE       HPQ       JPM      MSFT         V       WMT   
0          -0.000583 -0.034284 -0.015488 -0.038799 -0.037713 -0.006059   
1           0.018895  0.039103  0.035574  0.044549  0.041308  0.005331   
2           0.060457  0.010123 -0.000681 -0.000666  0.016342  0.010911   
3          -0.022243  0.012023 -0.003183  0.005331  0.003705  0.006092   
4          -0.009219 -0.000559 -0.002978  0.012325  0.010097 -0.004120   

                                
TickString         X       XOM  
0           0.000190 -0.015446  
1           0.102991  0.036737  
2           0.006829  0.005277  
3           0.014256  0.007218  
4           0.008706  0.005210

In [333]:
# Convert to a matrix.
matrixD = tickerD.to_numpy()

In [334]:
# Calculate the Covariance Matrix.
Sigma = (matrixD.transpose() @ matrixD)
Sigma = Sigma * (1/len(matrixD))

In [335]:
# Calculate the Eigenvalues and Eigenvectors 
L, Q = linalg.eig(Sigma)

In [336]:
# Percentage of variance explained by eigenvectors.
(L / L.sum()).cumsum()

array([0.37966589, 0.66018475, 0.74639596, 0.80751563, 0.85903163,
       0.89430168, 0.92292823, 0.94410825, 0.96065519, 0.96388066,
       0.96930749, 0.97824748, 0.98962165, 1.        ])

In [337]:
independent = np.array([eigenVectorP1, eigenVectorP2]).transpose()
dependent = np.array(matrixD.transpose()[2])

In [340]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# print("Number of Securities: " + str(tickerD.shape[1]))
# print("Number of Days: " + str(tickerD.shape[0]))

# Create the 'Eigenvector Portfolios'
eigenVectorP1 = matrixD @ Q[1] # Principal Component 1
eigenVectorP2 = matrixD @ Q[2]
eigenVectorP3 = matrixD @ Q[3]
eigenVectorP4 = matrixD @ Q[4]
eigenVectorP5 = matrixD @ Q[5]
eigenVectorP6 = matrixD @ Q[6]
eigenVectorP7 = matrixD @ Q[7]
eigenVectorP8 = matrixD @ Q[8]
eigenVectorP9 = matrixD @ Q[9]
eigenVectorP10 = matrixD @ Q[10]


independent = np.array([eigenVectorP1, eigenVectorP2]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 2 principal components: " + str(MSE))

independent = np.array([eigenVectorP1, eigenVectorP2, eigenVectorP3, eigenVectorP4, eigenVectorP5]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 5 principal components: " + str(MSE))

independent = np.array([eigenVectorP1, eigenVectorP2, eigenVectorP3, eigenVectorP4, eigenVectorP5, eigenVectorP6,
                        eigenVectorP7, eigenVectorP8]).transpose()
dependent = np.array(matrixD.transpose()[2])
# regressionData = regressionData.to_numpy()

# Regression Residuals for Microsoft
fittedmodel = model.fit(independent, dependent)
dependentNew = dependent.reshape(-1,1)
MSE = ((dependent - fittedmodel.predict(independent)) ** 2).sum()
print("Mean Squared Error using 8 principal components: " + str(MSE))


Mean Squared Error using 2 principal components: 0.0308211253704526
Mean Squared Error using 5 principal components: 0.025061957247186083
Mean Squared Error using 8 principal components: 0.01764812422086413


In [341]:
pwd

'C:\\Users\\AD'